<a href="https://colab.research.google.com/github/luizcalabria/redes_neurais_pos/blob/main/Otimiza%C3%A7%C3%A3o_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!git clone https://github.com/valmirf/redes_neurais_pos.git

fatal: destination path 'redes_neurais_pos' already exists and is not an empty directory.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation. 

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [ ]:
import random
import numpy as np

np.random.seed(10)
random.seed(10)

A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [ ]:
# Classe Network
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)  #número de neurônios em cada camada
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] #limiar
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] #pesos

    def feedforward(self, x):
        """Retorna a saída da rede z se `x` for entrada."""
        for b, w in zip(self.biases, self.weights):
            x = sigmoid(np.dot(w, x)+b) #net = (∑wx+b)
        return x

    def SGD(self, training_data, epochs, mini_batch_size, 𝜂, test_data=None, lmbd=0, otimizacao=None, momentum=0.8):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente. 
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""

        #dataset de treino
        training_data = list(training_data)
        n = len(training_data)
        
        #dataset de teste
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            #técnica que realiza o treinamento por lotes
            #mini_batch_size = tamanho do lote
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, 𝜂, n, lmbd, otimizacao)
            
            if test_data:
                acc = self.evaluate(test_data)
                print("    Epoch {} : {} / {} = {}%".format(j,acc,n_test,(acc*100)/n_test));
   
            else:
                print("    Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, 𝜂, len_training_data, lmbd=0, otimizacao=None, momentum=0.8):
        """Atualiza os pesos e limiares da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `a` é a taxa de aprendizado."""
        #inicializa matriz com derivadas de pesos e limiares
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, y in mini_batch:
          #resultado dos deltas do backpropagation sem a multiplicação da taxa de aprendizagem
          #soma os deltas do minibatch
          delta_nabla_b, delta_nabla_w = self.backprop(x, y)
          nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
          nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]

        #atualiza pesos e limiares (𝜂*𝛿*f’(net)*𝑥)
        n = len_training_data
        mb = len(mini_batch)
        if otimizacao=="L1":
          #https://github.com/meetvora/mlp-classifier/blob/master/neuralnet.py
          self.weights = [w-𝜂*lmbd*np.sign(w)/n-(𝜂/mb)*nw for w, nw in zip(self.weights, nabla_w)]
          self.biases = [b-(𝜂/mb)*nb for b, nb in zip(self.biases, nabla_b)]
        elif otimizacao=="L2":
          #https://visualstudiomagazine.com/articles/2017/09/01/neural-network-l2.aspx (An Alternative Approach)
          #self.weights = [(1-(𝜂/mb)*(lmbd/mb))*w-(𝜂/mb)*nw for w, nw in zip(self.weights, nabla_w)]
          self.weights = [(1-𝜂*(lmbd/n))*w-(𝜂/mb)*nw for w, nw in zip(self.weights, nabla_w)]
          self.biases = [b-(𝜂/mb)*nb for b, nb in zip(self.biases, nabla_b)]
        elif otimizacao=="MOMENTO":
          #https://github.com/Kanav123/MNIST-Neural-Network/blob/bb0fad846c6193f1272fba496a26217e589467d0/network2.py
          vel_w = [np.zeros(w.shape) for w in self.weights]
          vel_b = [np.zeros(b.shape) for b in self.biases]
          vel_w = [(momentum*v) - (𝜂/mb)*nw for v, nw in zip(vel_w, nabla_w)]
          vel_b = [(momentum*v) - (𝜂/mb)*nb for v, nb in zip(vel_b, nabla_b)]
          self.weights = [(v + (1- 𝜂*(lmbd/n))*w) for w ,v in zip(self.weights, vel_w)]
          self.biases = [(b + v) for b, v in zip(self.biases, vel_b)]
        else:
          self.weights = [w-(𝜂/mb)*nw for w, nw in zip(self.weights, nabla_w)]
          self.biases = [b-(𝜂/mb)*nb for b, nb in zip(self.biases, nabla_b)]


    def backprop(self, x, y):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo J_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        
        # Feedforward
        activation = x

        # Lista para armazenar todas as saídas dos neurônios (z), camada por camada
        activations = [x] 

        # Lista para armazenar todos os vetores net, camada por camada
        nets = [] 

        for b, w in zip(self.biases, self.weights):
            net = np.dot(w, activation)+b   
            nets.append(net)
            activation = sigmoid(net) #z = valor de saída do neurônio
            activations.append(activation)
        
        # Backward pass 
        
        #última camada -(u-z)f'(net)
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(nets[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) #(𝑦−𝑧)*f’(net)*𝑥
        
        # l = 1 significa a última camada de neurônios, l = 2 é a penúltima e assim por diante. 
        for l in range(2, self.num_layers):
            net = nets[-l]
            zs = sigmoid_prime(net)
            #delta da camada intermediaria. Note que utiliza o delta calculado anteriormente
            delta = np.dot(self.weights[-l+1].transpose(), delta) * zs 
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose()) #∑(𝛿𝑤)f’(net)𝑥
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Retorna o número de entradas de teste para as quais a rede neural 
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""

        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)

# Função de Ativação Sigmóide
def sigmoid(net):
    return 1.0/(1.0+np.exp(-net))

# Função para retornar as derivadas da função Sigmóide
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))


Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [ ]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('redes_neurais_pos/MLP/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (𝜂)




In [6]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)
test_data = list(test_data)

𝜂 = 0.3
arquitecture = [784, 40, 10] 
print("\n1. NET={}, ETA={}".format(arquitecture,𝜂))
print("  MLP")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data)

print("  otimizacao='L1'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.01, otimizacao="L1")

print("  otimizacao='L2'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.10, otimizacao="L2")

print("  otimizacao='MOMENTO'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.10, otimizacao="MOMENTO", momentum=0.8)



1. NET=[784, 40, 10], ETA=0.3
  MLP
    Epoch 0 : 4558 / 10000 = 45.58%
    Epoch 1 : 6105 / 10000 = 61.05%
    Epoch 2 : 6840 / 10000 = 68.4%
    Epoch 3 : 7388 / 10000 = 73.88%
    Epoch 4 : 7626 / 10000 = 76.26%
    Epoch 5 : 7741 / 10000 = 77.41%
    Epoch 6 : 7838 / 10000 = 78.38%
    Epoch 7 : 8159 / 10000 = 81.59%
    Epoch 8 : 8661 / 10000 = 86.61%
    Epoch 9 : 8770 / 10000 = 87.7%
    Epoch 10 : 8843 / 10000 = 88.43%
    Epoch 11 : 8892 / 10000 = 88.92%
    Epoch 12 : 8923 / 10000 = 89.23%
    Epoch 13 : 8949 / 10000 = 89.49%
    Epoch 14 : 8972 / 10000 = 89.72%
    Epoch 15 : 9000 / 10000 = 90.0%
    Epoch 16 : 9020 / 10000 = 90.2%
    Epoch 17 : 9043 / 10000 = 90.43%
    Epoch 18 : 9055 / 10000 = 90.55%
    Epoch 19 : 9073 / 10000 = 90.73%
  otimizacao='L1'
    Epoch 0 : 3749 / 10000 = 37.49%
    Epoch 1 : 4693 / 10000 = 46.93%
    Epoch 2 : 5500 / 10000 = 55.0%
    Epoch 3 : 6256 / 10000 = 62.56%
    Epoch 4 : 6420 / 10000 = 64.2%
    Epoch 5 : 6504 / 10000 = 65.04%
    E

In [8]:
𝜂 = 0.5
arquitecture = [784, 30, 20, 10] 
print("\n2. NET={}, ETA={}".format(arquitecture,𝜂))

print("  MLP")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data)

print("  otimizacao='L1'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.01, otimizacao="L1")

print("  otimizacao='L2'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.10, otimizacao="L2")

print("  otimizacao='MOMENTO'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.10, otimizacao="MOMENTO", momentum=0.8)



2. NET=[784, 30, 20, 10], ETA=0.5
  MLP
    Epoch 0 : 5625 / 10000 = 56.25%
    Epoch 1 : 7184 / 10000 = 71.84%
    Epoch 2 : 8003 / 10000 = 80.03%
    Epoch 3 : 8373 / 10000 = 83.73%
    Epoch 4 : 8582 / 10000 = 85.82%
    Epoch 5 : 8691 / 10000 = 86.91%
    Epoch 6 : 8787 / 10000 = 87.87%
    Epoch 7 : 8855 / 10000 = 88.55%
    Epoch 8 : 8884 / 10000 = 88.84%
    Epoch 9 : 8952 / 10000 = 89.52%
    Epoch 10 : 8969 / 10000 = 89.69%
    Epoch 11 : 9018 / 10000 = 90.18%
    Epoch 12 : 9047 / 10000 = 90.47%
    Epoch 13 : 9070 / 10000 = 90.7%
    Epoch 14 : 9090 / 10000 = 90.9%
    Epoch 15 : 9092 / 10000 = 90.92%
    Epoch 16 : 9124 / 10000 = 91.24%
    Epoch 17 : 9139 / 10000 = 91.39%
    Epoch 18 : 9173 / 10000 = 91.73%
    Epoch 19 : 9184 / 10000 = 91.84%
  otimizacao='L1'
    Epoch 0 : 6357 / 10000 = 63.57%
    Epoch 1 : 7813 / 10000 = 78.13%
    Epoch 2 : 8302 / 10000 = 83.02%
    Epoch 3 : 8536 / 10000 = 85.36%
    Epoch 4 : 8658 / 10000 = 86.58%
    Epoch 5 : 8796 / 10000 = 87.9

In [ ]:
𝜂 = 0.3
arquitecture = [784, 30, 10]
print("\n3. NET={}, ETA={}".format(arquitecture,𝜂))

print("  MLP")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data)

print("  otimizacao='L1'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.01, otimizacao="L1")

print("  otimizacao='L2'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.10, otimizacao="L2")

print("  otimizacao='MOMENTO'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.10, otimizacao="MOMENTO", momentum=0.8)


3. NET=[784, 30, 10], ETA=0.3
  MLP
    Epoch 0 : 3120 / 10000 = 31.2%
    Epoch 1 : 4832 / 10000 = 48.32%
    Epoch 2 : 5578 / 10000 = 55.78%
    Epoch 3 : 6316 / 10000 = 63.16%
    Epoch 4 : 7263 / 10000 = 72.63%
    Epoch 5 : 7542 / 10000 = 75.42%
    Epoch 6 : 7678 / 10000 = 76.78%
    Epoch 7 : 7762 / 10000 = 77.62%
    Epoch 8 : 8624 / 10000 = 86.24%
    Epoch 9 : 8722 / 10000 = 87.22%
    Epoch 10 : 8806 / 10000 = 88.06%
    Epoch 11 : 8876 / 10000 = 88.76%
    Epoch 12 : 8909 / 10000 = 89.09%
    Epoch 13 : 8946 / 10000 = 89.46%
    Epoch 14 : 8963 / 10000 = 89.63%
    Epoch 15 : 8982 / 10000 = 89.82%
    Epoch 16 : 8993 / 10000 = 89.93%
    Epoch 17 : 9018 / 10000 = 90.18%
    Epoch 18 : 9022 / 10000 = 90.22%
    Epoch 19 : 9035 / 10000 = 90.35%
  otimizacao='L1'
    Epoch 0 : 4586 / 10000 = 45.86%
    Epoch 1 : 6987 / 10000 = 69.87%
    Epoch 2 : 7552 / 10000 = 75.52%
    Epoch 3 : 7743 / 10000 = 77.43%
    Epoch 4 : 7864 / 10000 = 78.64%
    Epoch 5 : 7970 / 10000 = 79.7%
 

##**Mini-Projeto**

A partir dos melhores resultados obtidos no projeto anterior, execute 3 configurações pra cada questão a seguir:


1) Implementar as regularizações L1 e L2

2) Implementar o Momento

3) Comparar os experimentos e explicar o porquê de cada resultado. Qual foi a melhor regularização? Por que? O Momento melhorou os resultados? Por que? 
<blockquote>
<p>
Neste mini-projeto foi utilizado os 3 melhores resultados do mini-projeto de MLP
<pre>
1º 𝜂 = 0.3, net = [784,40, 10] = 76.71% 
2º 𝜂 = 0.5, net = [784, 30, 20, 10] = 74.25%
3º 𝜂 = 0.3, net = [784, 30, 20, 10] = 89.94%
</pre>
.
Começo comparando cada cada execução com a do projeto anterior
<blockquote>
<p><b>Usando a 1º configuração:</b></p>
<p>
A utilização do L1 mostrou um resultado pior que até mesmo a execução no mini-projeto anterior atingindo apenas 74.22%, o melhor resultado atingido nesta configuração foi utilizando L2, chegando a 90.93%
</p>
<p><b>Usando a 2º configuração:</b></p>
<p>
Já nesta configuração todos os resultados foram melhores que o resultado do mini-projeto anterior sendo o melhor resultado obtido com a utilização do L1, chegando a 92.42% e tendo o o pior resultado utilizando L2
</p>
</p>
<p><b>Usando a 3º configuração:</b></p>
<p>
</blockquote>
O melhor resultado obtido foi: <strong>92.15%</strong>, na configuração 1 (𝜂 = 0.5, net = [784, 30, 20, 10]), utilizando o Momento. Mas é bom registrar que todos os resultados foram melhores que os originais.
</p>

</blockquote>

Complete a Tabela abaixo com os resultados 

\begin{array}{|c|c|c|c|}\hline\\ \\
  Configuracao & 1 & 2 & 3 \\ \hline 
L1 & 74.22 & 92.42 & 90.42  \\ \hline
L2  & 90.93 & 92.02 & 90.41 \\ \hline 
Momento  & 90.75 & 92.29 & 90.47 \\ \hline
\end{array}



\\
Data de Entrega: 23/12/2020
